# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [1]:
import pandas as pd
import numpy as np
import os

DATA_FOLDER = 'data/' # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

In [2]:
# Write your answer here
countries = dict.fromkeys(['guinea', 'liberia', 'sl'])

for country in countries:
    FULL_PATH = DATA_FOLDER + "ebola/" + country + "_data/"
    print("")
    for csv_file in os.listdir(FULL_PATH):
        if countries[country] is None:
            countries[country] = pd.read_csv(FULL_PATH + csv_file, sep=',')
            print(countries[country].shape[1], end=",")
        else:
            nextFrame = pd.read_csv(FULL_PATH + csv_file, sep=',')
            print(nextFrame.shape[1], end=",")
            countries[country] = pd.concat([countries[country], nextFrame], axis=0)

            
            


24,24,24,24,24,24,24,24,24,24,24,24,16,24,24,24,24,24,24,21,24,24,
18,18,18,18,16,18,16,16,13,18,18,16,18,18,18,18,14,17,18,16,17,18,18,14,18,19,15,18,15,13,18,14,18,17,16,14,18,18,14,15,18,18,18,14,16,14,18,18,18,18,17,18,18,18,18,14,14,18,14,18,14,14,14,18,18,18,18,18,17,17,15,18,18,16,18,18,18,18,14,18,18,18,14,18,18,16,18,17,14,14,18,18,17,14,18,18,18,18,18,14,
19,17,17,17,18,20,17,17,20,17,17,17,17,18,18,17,17,17,17,17,21,18,17,20,21,17,18,17,17,17,17,17,17,18,17,21,17,17,17,20,21,18,18,17,17,21,17,18,18,17,18,18,17,19,17,18,20,20,17,18,17,17,19,18,18,17,17,17,17,17,17,17,17,17,18,17,20,17,21,17,17,17,18,17,17,21,18,17,20,21,21,17,18,18,20,17,17,18,17,18,18,17,17,

We only consider confirmed cases as legitimate cases.

In [38]:
dt_info = {
    'guinea' : {
        'case' : 'New cases of confirmed',
        'deaths' : [
            'New deaths registered',
            'New deaths registered today'
        ],
        'var' : 'Description',
        'exclude' : ['Totals', 'Description'],
        'date' : 'Date',
        'dateformat' : '%Y-%m-%d'
    },
    'liberia' : {
        'case' : 'New case/s (confirmed)',
        'deaths' : [
            'Newly reported deaths'
        ],
        'var' : 'Variable',
        'exclude' : ['National', 'Variable'],
        'date' : 'Date',
        'dateformat' : '%m/%d/%Y'
    },
    'sl' : {
        'case' : 'new_confirmed',
        'deaths' : [
            'etc_new_deaths'
        ],
        'var' : 'variable',
        'exclude' : ['National', 'variable'],
        'date' : 'date',
        'dateformat' : '%Y-%m-%d'
    }
}
"""
 TODO:
    - investigate unnamed
    - check that variable columns in original data does not cause trouble
    - 
"""

cases = []
deaths = []
for country in countries:
    info = dt_info[country]
    df = countries[country]

    # filter out cases and deaths

    # parse date
    df[info['date']] = pd.to_datetime(df[info['date']])
    # set index
    df = df.set_index(info['date'])

    # sort by date
    df.sort_index(inplace=True)

    # get rid of NaN values
    df = df.fillna(0)
    # get rid of dashes
    df = df.replace('-', '')

    
    # filter out cases
    dfc = df.loc[df[info['var']] == info['case']].copy(deep=True)
    # filter out deaths
    dfd = df.loc[df[info['var']].isin(info['deaths'])].copy(deep=True)
    
    # delete non-numerics
    for column in info['exclude']:
        del dfc[column]
        del dfd[column]
        
    # convert all values to numeric
    for column in list(dfc):
        dfc[column] = pd.to_numeric(dfc[column])
        dfd[column] = pd.to_numeric(dfd[column])
        
    # collapse regions
    dfc[country] = dfc[list(dfc)].sum(axis=1)
    dfc = dfc[country]
    dfd[country] = dfd[list(dfd)].sum(axis=1)
    dfd = dfd[country]
    
    # exclude 4th to 9th december in liberia (wrong data, total instead of new cases)
    if country == 'liberia':
        dfc.loc['2014-12-04':'2014-12-09'] = 0
    # create daily average per month
    dfc = dfc.groupby(pd.TimeGrouper(freq='M')).mean()
    cases.append(dfc)
    dfd = dfd.groupby(pd.TimeGrouper(freq='M')).mean()
    deaths.append(dfd)
    

#header_c = pd.MultiIndex.from_product(
#    [list(countries), ['cases', 'deaths']
#)

dfc = pd.concat(cases, axis=1)
dfd = pd.concat(deaths, axis=1)
pd.concat([dfc, dfd], axis=1)

,guinea,liberia,sl,guinea,liberia,sl
Date,,,,,,
2014-06-30,NaN,2.142857,NaN,NaN,1.857143,NaN
2014-07-31,NaN,1.818182,NaN,NaN,4.272727,NaN
2014-08-31,12.4000,5.444444,19.600000,3.2000,23.222222,5.550000
2014-09-30,12.5625,6.166667,36.275862,3.5625,36.041667,4.344828
2014-10-31,6.0000,1.240000,57.535714,15.0000,28.120000,2.071429
2014-11-30,NaN,2.533333,63.238095,NaN,13.666667,0.428571
2014-12-31,NaN,1.111111,32.600000,NaN,0.000000,0.000000


## Task 2. RNA Sequences  In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each.  
Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

In [ ]:
# Write your answer here

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
from IPython.core.display import HTML
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
# Write your answer here